# From Parsed to Embedding

In [37]:
import pandas as pd
from leomodules.parsing import parsers
# pdf = parsers.parse_mupdf("Transcripts_English.pdf", pages=range(1,100), sentecizer=parsers.spacy_sentecizer)
pdf = pd.read_csv('parsed_transcripts.csv', index_col=0)

pdf = pdf.query('page>40')

texts = sum([' ' + pdf.text.shift(-i) for i in range(1,4)], pdf.text).astype(str)

text_sample = texts.sample(10_000)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI

from langchain.vectorstores import FAISS



# from leomodules.mysecrets import OPENAI_API_KEY
# os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
# from leomodules.embedding import create_vectorstore

create_vectorstore(text_sample.to_list())

# Not Parsing

In [9]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
import os
from langchain.vectorstores import FAISS

from leomodules.mysecrets import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

embeddings = OpenAIEmbeddings()

In [3]:
from langchain.vectorstores import FAISS
index = FAISS.load_local("indexes", embeddings)

In [11]:
# TODO extract the transcript from the website, since it is in paragraphs (see how to extract links from pdfs)
# TODO remove the titles

In [5]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=index.as_retriever(), return_source_documents=True)

In [6]:
query = 'I feel very stressed.'
result = qa({'query':query})
for k, v in result.items():
    print(k, '-'*len(k), sep='\n')
    if not isinstance(v, list):
        print(v)
        print('\n')
    else:
        for doc in v:
            print(f'"{doc.page_content}"')
        print('\n')
    

query
-----
I feel very stressed.


result
------
 It can be difficult to feel stressed by the behavior of others. It sounds like you have been through a lot and have taken steps to let the other people know how you are feeling. If you need help managing your stress, it might be a good idea to talk to someone like a counselor or therapist.


source_documents
----------------
"I really feel very hurt. But I must tell you the headaches [that] London Sahaja Yogis and English Sahaja Yogis have given me, as nobody has given me such headaches. I have worked hardest in this place."
"I don’t like that kind of things. No mother would like, I don’t feel peaceful with you, I fell very nervous. Alright? So you understand My concern about you and I mean it, I really mean it."
"What should I do?" I feel so helpless in America. But those who are Sahaja Yogis here, are great Sahaja Yogis, I must say. They are wonderful."
"I don't know my fate is bad I got such horrible people. If you are thinking of .

In [7]:
index.similarity_search_with_relevance_scores('Find a solution for: "I feel stressed."')

[(Document(page_content='What’s the use of feeling bad? Isn’t it? You must solve the problem. Not to feel sorry, but to do it.', metadata={}),
  0.7501563690865432),
 (Document(page_content='On the contrary they wanted to improve it and work it out. But there’s a very simple way of doing it. You don’t have to go out of the way to change people or to make any sacrifices, nothing of the kind. Everybody today is suffering from some sort of anxiety, some sort of problems, some sort of worries.', metadata={}),
  0.7393457235169232),
 (Document(page_content='First thing you will feel is peace. It is as if you are standing in water and you say hope we do not drown in these thoughts. You say you have a lot of questions. But fortunately if a boat arrives you will climb into it and that fear of drowning is gone from you.', metadata={}),
  0.737609821267305),
 (Document(page_content='He feels that brother has come and, "What should be done to make him feel good? What to give? How to give rest? Wh

In [60]:
index.similarity_search_with_relevance_scores('Find the opposite of: "I feel stressed."')

[(Document(page_content='And unless and until you have connected with God, whatever you may try, whatever you may acquire, you can never have that joy. You may have that artificial confidence, because a person who is frustrated has to say - show outside that - "Oh no, I\'m not frustrated. I am very happy." It\'s like to be tense and say, "I\'m very relaxed" - it\'s like that.', metadata={}),
  0.719120019899),
 (Document(page_content='You have to fight your negativity, because negativity is the thing that asks questions. When I am talking I am telling the truth, absolute truth, but it asks questions and it reflects. When it starts reflecting, nothing goes in the head because you are left with the past sentence, and the present you are not with it. So the whole thing boils down to something like an escape, and then you escape and you sleep off.', metadata={}),
  0.7095963494220843),
 (Document(page_content='Put both your feet apart from each other. Be comfortable. Now start feeling a ki

## LLM-assisted retriever

In [10]:
from langchain.retrievers.multi_query import MultiQueryRetriever
import logging
from langchain.chat_models import ChatOpenAI

question = """
"I feel stressed."
"""

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=index.as_retriever(), llm=llm)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can I reduce stress levels?', '2. What are some effective ways to manage stress?', '3. Are there any techniques or strategies to alleviate stress?']


8

In [11]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever_from_llm, return_source_documents=True)

In [12]:
query = 'I feel very stressed.'
result = qa({'query':query})
for k, v in result.items():
    print(k, '-'*len(k), sep='\n')
    if not isinstance(v, list):
        print(v)
        print('\n')
    else:
        for doc in v:
            print(f'"{doc.page_content}"')
        print('\n')
    

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can I reduce stress levels effectively?', '2. What are some strategies to manage and alleviate stress?', '3. Are there any proven techniques to help cope with stress?']


query
-----
I feel very stressed.


result
------
 Practicing Sahaja Yoga can help you reduce stress. Relax in a peaceful atmosphere, take time to rest, read calming books, and focus on your spiritual growth. Trust yourself and have faith in yourself. Observe your own ego and address yourself as if you are looking in a mirror. Enjoy the beauty of nature, and allow yourself to be receptive and not complaining. Finally, take salt treatment to reduce stress.


source_documents
----------------
"And to be tense is out of, out of absolutely we can say, is out of subject, is useless. How can you be tense in the atmosphere which is so relaxed? If these things are expressing themselves, then you should know you haven’t come to your proper point in Sahaja Yoga. If you still worry and go on worrying, you’ll go to the left and you’ll get out of Sahaja Yoga."
"At the time when the mother is pregnant, she should not work very hard. She should take rest more. She should not think too much but read s

In [13]:
query = 'My wife is very rude with me.'
result = qa({'query':query})
for k, v in result.items():
    print(k, '-'*len(k), sep='\n')
    if not isinstance(v, list):
        print(v)
        print('\n')
    else:
        for doc in v:
            print(f'"{doc.page_content}"')
        print('\n')
    

INFO:langchain.retrievers.multi_query:Generated queries: ["1. How can I deal with my wife's rude behavior towards me?", "2. What strategies can I use to address my wife's disrespectful attitude towards me?", '3. How can I improve the communication and relationship with my wife, considering her rude behavior towards me?']


query
-----
My wife is very rude with me.


result
------
 You should try to be understanding and patient with your wife. Try to be diplomatic and humorous or use small gestures to show her that you care. Teach your children to respect their mother and avoid scolding her in front of them. Try to focus on the sweet things in your relationship and avoid getting too hung up on minor issues.


source_documents
----------------
"We should have all decency of husband and wife, there are so many things we do not do in the presence of our children. In the same way we need not sit down and quarrel and say: 'I hate you and I hate you'. I tell you this is something you have to change little bit in our language. In India if somebody says: 'I hate you' it's regarded, that man regarded as a most unmannerly person."
"do this ….do that…wear this, behave like this……etc and shows her finger with order to him……then he becomes silent and obeys. Husband has all the right to correct his wife, but with love,

In [70]:
query = 'I feel sad and depressed.'
result = qa({'query':query})
for k, v in result.items():
    print(k, '-'*len(k), sep='\n')
    if not isinstance(v, list):
        print(v)
        print('\n')
    else:
        for doc in v:
            print(f'"{doc.page_content}"')
        print('\n')
    

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can I overcome feelings of sadness and depression?', '2. What are some strategies to cope with sadness and depression?', '3. Are there any effective methods to alleviate sadness and depression?']


query
-----
I feel sad and depressed.


result
------
 It's okay to feel sad and depressed sometimes. It is important to recognize and accept these feelings and to take care of yourself. It is important to build a strong sense of self-love and self-worth. To do this, practice self-care activities and nurture your relationships with yourself and others. Additionally, try to focus on positive things and practice gratitude.


source_documents
----------------
"The main point is to have faith in yourself. In this short time, I cannot tell you more than this but what I have to tell you that above all you jump into the ocean of joy. Happiness and unhappiness are the two sides of one coin. When your ego is pampered you feel happy."
"This is the first step of introspection. Now what helps for this introspection is your sense of love. Can you love a person who is distressful? One who is aggressive?"
"What’s the use of feeling bad? Isn’t it? You must solve the problem. Not to feel sorry, but to 

In [ ]:
# Seeker/yogi writes a query (what he is feeling)
# query -> fed to chatGPT (which creates new queries to searched) -> vector(s) -> semantic search in a database -> seeker/yogi is fed with the best results

# TODO: instead of chatGPT, we can make standard queries
# TODO: change the database: instead of 4-sentences and all talks -> use paragraphs and only PP and 
# TODO: Graph-index
# TODO: filter the talks to be used

